In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
document.title='AiiDAlab SSSP app'

<IPython.core.display.Javascript object>

In [2]:
%matplotlib widget

In [3]:
import ipywidgets as ipw
from pathlib import Path
from IPython.display import display
import matplotlib.pyplot as plt
import os
import json

from aiidalab_sssp.inspect.subwidgets.periodic_table import PeriodicTable
from aiidalab_sssp.inspect.subwidgets.select import PseudoSelectWidget
from aiidalab_sssp.inspect.subwidgets.summary import SummaryWidget
from aiidalab_sssp.inspect.subwidgets.delta import AccuracyMeritWidget, EosComparisonWidget
from aiidalab_sssp.inspect.subwidgets.bands import BandStructureWidget, BandChessboard
from aiidalab_sssp.inspect.subwidgets.convergence import ConvergenceWidget


from aiidalab_sssp.inspect import DB_FOLDER, SSSP_LOCAL_DB

# Show plot one time dynamical
plt.ioff()

# If cached DB folder not exist (first time app run), create folder
DB_FOLDER.mkdir(parents=True, exist_ok=True)
SSSP_LOCAL_DB.mkdir(parents=True, exist_ok=True)

ptable = PeriodicTable(cache_folder=str(DB_FOLDER))
pseudo_select = PseudoSelectWidget()
summary = SummaryWidget()

eos_comparison = EosComparisonWidget()

nu_preview = AccuracyMeritWidget()
bandchessboard = BandChessboard()
convergence = ConvergenceWidget()
bandstucture = BandStructureWidget()

ipw.dlink(
    (ptable, 'pseudos'),
    (pseudo_select, 'pseudos'),
)

####################
# load process from url 
# this need to happened before link to pseudos of following widgets
from urllib.parse import parse_qs, urlsplit

def parse_url(url):
    query = parse_qs(urlsplit(url).query)
    pk = query.get('pk', None)

    # Can be a list with multiple nodes to load
    return pk

def _load_pseudos(element, db=SSSP_LOCAL_DB) -> dict:
    """Open result json file of element return as dict"""
    if element:
        json_fn = os.path.join(db, f"{element}.json")
        with open(json_fn, "r") as fh:
            pseudos = json.load(fh)

        return {key: pseudos[key] for key in sorted(pseudos.keys(), key=str.lower)}

    return dict()

try:   
    pk = int(parse_url(jupyter_notebook_url)[0])
except Exception:
    pk = None

if pk:
    import aiida
    from aiida.orm import load_node
    from aiidalab_sssp.inspect import dump_to_sssp_local_db

    aiida.load_profile()

    node = load_node(pk)
    label = node.extras.get("label").split()[-1]
    element = node.extras.get("element")

    try:
        pseudos = _load_pseudos(element)
    except FileNotFoundError:
        pseudos = {}

    if label not in pseudos:
        curated_results = pseudos
        # dump bands/band structure and return a dict of pseudo
        tmp_pseudo = dump_to_sssp_local_db(node)
        pseudos[label] = tmp_pseudo
        
    ptable.selected_element = element
    ptable.ptable.selected_elements = {element: 0}  # TODO: this should be in ptabel widget
    pseudo_select.selected_pseudos[f'{label}(custom)'] = pseudos[label]

    # TODO: add a custom instruction as output and chime into after the ptable
    # to let user know that the local verified pseudo is to compare here.
########################


ipw.dlink((pseudo_select, 'selected_pseudos'), (summary, 'pseudos'))
ipw.dlink((pseudo_select, 'selected_pseudos'), (nu_preview, 'pseudos'))
ipw.dlink((pseudo_select, 'selected_pseudos'), (eos_comparison, 'pseudos'))
ipw.dlink((pseudo_select, 'selected_pseudos'), (bandchessboard, 'pseudos'))
ipw.dlink((pseudo_select, 'selected_pseudos'), (bandstucture, 'pseudos'))
ipw.dlink((pseudo_select, 'selected_pseudos'), (convergence, 'pseudos'))


display(ptable)
display(pseudo_select)
display(nu_preview)
display(summary)
display(eos_comparison)
display(bandchessboard)
display(bandstucture)
display(convergence)

PeriodicTable(children=(PTableWidget(allElements=['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', …

PseudoSelectWidget(children=(HTML(value='No pseudopotentials available for compare, please select an element o…

AccuracyMeritWidget(children=(Output(),), layout=Layout(display='none'))

SummaryWidget(children=(Output(), HTML(value='<p> Switch between ν and Δ </p>'), ToggleButtons(options=(('ν', …

EosComparisonWidget(children=(HBox(children=(Dropdown(options=(), value=None), Dropdown(options=(), value=None…

BandChessboard(children=(HTML(value='<h2> Accuracy: Bands distance chessboard</h2>'), Output()), layout=Layout…

BandStructureWidget(children=(HTML(value='<h2> Band Structure </h2>'), HBox(children=(Dropdown(options=(), val…